In [11]:
%%capture
!git clone https://github.com/ronakdm/input-marginalization.git

In [12]:
%%bash
cd input-marginalization
git pull
cd ..

Updating 37d0591..b24eb82
Fast-forward
 generative_model/data/wikitext-2/test.txt    |  4358 ---
 generative_model/data/wikitext-2/train.txt   | 36718 -------------------------
 generative_model/data/wikitext-2/valid.txt   |  3760 ---
 generative_model/dataset.py                  |    66 -
 generative_model/test.py                     |    58 -
 generative_model/transformer-wikitext2.ipynb |     2 +-
 6 files changed, 1 insertion(+), 44961 deletions(-)
 delete mode 100644 generative_model/data/wikitext-2/test.txt
 delete mode 100644 generative_model/data/wikitext-2/train.txt
 delete mode 100644 generative_model/data/wikitext-2/valid.txt
 delete mode 100644 generative_model/dataset.py
 delete mode 100644 generative_model/test.py


From https://github.com/ronakdm/input-marginalization
   37d0591..b24eb82  main       -> origin/main


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
%%capture
!pip install transformers

In [15]:
import os,sys,time,math,textwrap

import numpy as np
import torch
import torch.nn as nn

sys.path.append("input-marginalization/generative_model")
# import dataset, transformer
import transformer
from gen_model_utils import WikiText2Dataset

# root = 'input-marginalization/generative_model/data/wikitext-2'
root = 'input-marginalization/generative_model/data/wikitext-2-raw'

In [16]:
lr = .00035
context = 150
batch_size = 32
log_interval = 50

heads = 1
depth = 1

torch.manual_seed(0)
device = torch.device("cuda")

In [17]:
# train_data = dataset.WikiText2(root, context, dataset.DatasetSplit.train)
# valid_data = dataset.WikiText2(root, context, dataset.DatasetSplit.valid)
# test_data = dataset.WikiText2(root, context, dataset.DatasetSplit.test)

train_data = WikiText2Dataset(file_path=root+"/wiki.train.raw", seq_len=context)
valid_data = WikiText2Dataset(file_path=root+"/wiki.valid.raw", seq_len=context)
test_data = WikiText2Dataset(file_path=root+"/wiki.test.raw", seq_len=context)

Token indices sequence length is longer than the specified maximum sequence length for this model (2303697 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (238658 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (273180 > 512). Running this sequence through the model will result in indexing errors


In [18]:
def evaluate(data):
    model.eval()
    with torch.no_grad():
        loss = 0.
        loader = torch.utils.data.DataLoader(dataset=data,batch_size=batch_size,shuffle=False)
        for i, (x,y) in enumerate(loader):
            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            yhat = model(x).view(-1, train_data.word_count())
            loss += criterion(yhat, y.contiguous().view(-1))

    print()
    model.train()
    return loss / len(loader)

In [19]:
# Real params. TODO: Change back.
# d = 400  # Word embedding dimension.
# k = 40   # Key, query, and value dimension
# m = 900  # Linear layer dimension

d = 200  # Word embedding dimension.
k = 40   # Key, query, and value dimension
m = 500  # Linear layer dimension

# Test params.
# d = 100  # Word embedding dimension.
# k = 20   # Key, query, and value dimension
# m = 200  # Linear layer dimension

model = transformer.Transformer(context, train_data.word_count(), d, k, m, heads, depth, tied_weights=True).to(device)
count = sum([np.prod(parm.shape) for parm in model.parameters() if parm.requires_grad])
print('Initialized graph with {} parameters'.format(count))

Initialized graph with 3146321 parameters


In [20]:
criterion = nn.NLLLoss()
curr_lr = .0001
clip = .25
best_val_loss = None
epochs = 10 # TODO: Change to 10 or 20.
# save = 'model.pt'
save_filename = 'transformer_wikitext2'

train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
print('Initiating training, {} iterations/epoch.'.format(len(train_loader)))

try:
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)
    for epoch in range(epochs):
        t0 = time.time()
        val_loss = evaluate(valid_data)
        print('-' * 100)
        print('| checkpoint | epoch {:3d} | time: {:5.2f}s | validation loss {:5.2f} | '
                'validation perplexity {:8.2f}'.format(epoch, (time.time() - t0),
                                                       val_loss, math.exp(val_loss)))
        print('-' * 100)
        print('epoch\t\tms/batch\tlr\tloss\tperplexity')

        if not best_val_loss or val_loss < best_val_loss:
            # with open(save, 'wb') as f:
            #     torch.save(model, f)
            torch.save(model, f"{save_dir}/{save_filename}.pt") # imarg: Save in Drive folder.
            best_val_loss = val_loss

        model.train()
        total_loss = 0.
        t0 = time.time()
        if epoch == 1: optimizer.param_groups[0]['lr'] = curr_lr = lr # finished warmup
        for i, (x,y) in enumerate(train_loader):
            if i % log_interval == 0 and i > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - t0
                print('{:3d} ({:2.1f}%)\t{:5.2f}\t\t{:1.3}\t{:5.2f}\t{:8.2f}'.format(
                    epoch, 100*i/float(len(train_loader)),
                    elapsed * 1000 / log_interval, curr_lr, cur_loss, math.exp(cur_loss)))
                total_loss = 0
                t0 = time.time()

            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            model.zero_grad()
            yhat = model(x).view(-1, train_data.word_count())
            loss = criterion(yhat, y.contiguous().view(-1))
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            total_loss += loss.item()

except KeyboardInterrupt:
    print('Graceful Exit')

Initiating training, 480 iterations/epoch.

----------------------------------------------------------------------------------------------------
| checkpoint | epoch   0 | time:  1.26s | validation loss 10.34 | validation perplexity 30981.03
----------------------------------------------------------------------------------------------------
epoch		ms/batch	lr	loss	perplexity
  0 (10.4%)	60.90		0.0001	10.27	28719.75
  0 (20.8%)	61.18		0.0001	 9.90	19972.91
  0 (31.2%)	60.96		0.0001	 9.28	10743.06
  0 (41.7%)	60.96		0.0001	 8.79	 6599.99
  0 (52.1%)	60.89		0.0001	 8.40	 4428.13
  0 (62.5%)	60.75		0.0001	 8.04	 3101.05
  0 (72.9%)	60.96		0.0001	 7.74	 2306.86
  0 (83.3%)	60.88		0.0001	 7.50	 1804.13
  0 (93.8%)	61.64		0.0001	 7.33	 1531.00


In [21]:
print('Restoring best checkpointed model...')
# with open(save, 'rb') as f:
#     model = torch.load(f)
model = torch.load(f"{save_dir}/{save_filename}.pt")

test_loss = evaluate(test_data)
print('=' * 89)
print('| end of training | test loss {:5.2f} | test perplexity {:8.2f}'.format(test_loss, math.exp(test_loss)))
print('=' * 89)

Restoring best checkpointed model...

| end of training | test loss 10.34 | test perplexity 30976.45


In [38]:
print('\nUncurated samples')
print('-' * 89)

def sample():
    # words = []
    model.eval()

    ids = []
    history = torch.randint(train_data.word_count(), (1, 1), dtype=torch.long).cuda()
    for i in range(context):
        output = model(history)
        word_weights = output[-1].squeeze().exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        word_tensor = torch.Tensor([[word_idx]]).long().cuda()
        history = torch.cat([history, word_tensor], 0)

        # words.append(train_data.idx2word[word_idx])
        ids.append(int(word_idx))

    # return '\n'.join(textwrap.wrap(' '.join(words),80))
    return tokenizer.decode(ids)

for i in range(5):
    print('({})'.format(i), sample())


Uncurated samples
-----------------------------------------------------------------------------------------
(0) mecca shortages 257 coca uptown favourable magnateis magic priestess improved broadlynus [unused542] gardening presiding 武ackerₕ ib barron noon involving earliest liب mai doneamps wrought intimately shelby arthurings cheated [unused133]elis wonder ions [unused784] utrecht robertoticsอ metz welfareeiro micahkill regulate relatives registrar 1801 warn heel vocalcala boar exclude commemoratelovaa [unused40]notenting granville fled obligation degraded beds exited siblings properly gothenburg [unused491] mallory／path cincinnati blog securinginator llneck horizon shore incentives tremor carole paranoia mortimer mesopotamia perspectivecumдtern gaspingsha fallon nonetheless damaging plant mountains associate forceful conclusions shanghainam recognizes chichesterega authoritarian nestor investigative tightly prompting regulatesior boston [unused442] stroked egyptiantailvna panama 306